# Unity Catalog Setup — External Locations, Volumes, Catalog & Schema

This notebook establishes the **foundational governance layer** for the Rearc Quest lakehouse using **Databricks Unity Catalog**.

It configures:
- External Locations (cloud storage access boundary)
- Permissions on those locations
- Catalog and schema structure
- Volumes for governed file-based access

This setup ensures that **all downstream ingestion and analytics notebooks operate under a secure, auditable, and scalable data governance model**.

---

In Unity Catalog:
- **External Locations** define *where* data can physically live
- **Grants** define *who* can read/write/manage that storage
- **Catalogs & Schemas** define *logical organization*
- **Volumes** provide *governed file access* without bypassing Unity Catalog controls

---

## Design principles applied

- **Least privilege access**: permissions are granted explicitly and scoped
- **Separation of concerns**:
  - Storage access (external locations)
  - Logical organization (catalog / schema)
  - Data access (tables, volumes)
- **Production-ready**: supports multi-user access, auditability, and future expansion

---

## Resulting hierarchy

#### External Location (cloud storage)
#####└── Catalog: rearc_quest
#####└── Schema: lakehouse
#####├── Volumes (raw file access)
#####├── Managed tables (analytics outputs)



# External Location

In [0]:
%sql 

-- Inspect all external locations configured in the workspace.
-- This is a sanity check to confirm the storage boundary already exists
-- and to avoid duplicate or conflicting definitions.

show external locations

name,url,comment
dltdemo,abfss://landing@rohitstorage1985.dfs.core.windows.net/landing_data,
rearc_quest,abfss://rearc-quest@rohitstorage1985.dfs.core.windows.net/,
rohit1985_dbx,abfss://unity-catalog-storage@dbstorageov5dultuqlpj4.dfs.core.windows.net/1989428863478772,null
rohit_data,abfss://data-ro@rohitstorage1985.dfs.core.windows.net/data_folder,null
rohit_external,abfss://data-ro@rohitstorage1985.dfs.core.windows.net/external,external location for handling external data in adls
rohit_managed,abfss://data-ro@rohitstorage1985.dfs.core.windows.net/data_folder_managed,null


In [0]:
%sql

-- Review current grants on the external location.
-- Important for validating least-privilege access and debugging permission issues
-- before attempting volume creation or file writes.

Show grants on external location rearc_quest;

Principal,ActionType,ObjectType,ObjectKey


In [0]:
%sql

-- Allows the principal to create managed objects (e.g., volumes, managed tables)
-- backed by this external location.
-- This does NOT grant read/write file access by itself.

GRANT CREATE MANAGED STORAGE ON EXTERNAL LOCATION rearc_quest TO `ro.azure.1985_gmail.com#EXT#@roazure1985gmail.onmicrosoft.com`;

In [0]:
%sql

-- Grants low-level file access on the external location.
-- Required for:
--  - writing raw JSON/text files
--  - using dbutils.fs with volumes
--  - external file-based ingestion patterns

GRANT READ FILES, WRITE FILES ON EXTERNAL LOCATION rearc_quest TO `ro.azure.1985_gmail.com#EXT#@roazure1985gmail.onmicrosoft.com`;


In [0]:
%sql
Show grants on external location rearc_quest;

Principal,ActionType,ObjectType,ObjectKey
ro.azure.1985_gmail.com#ext#@roazure1985gmail.onmicrosoft.com,CREATE MANAGED STORAGE,EXTERNAL LOCATION,rearc_quest
ro.azure.1985_gmail.com#ext#@roazure1985gmail.onmicrosoft.com,READ FILES,EXTERNAL LOCATION,rearc_quest
ro.azure.1985_gmail.com#ext#@roazure1985gmail.onmicrosoft.com,WRITE FILES,EXTERNAL LOCATION,rearc_quest


In [0]:
%sql

GRANT USAGE ON CATALOG rearc_quest TO `ro.azure.1985_gmail.com#EXT#@roazure1985gmail.onmicrosoft.com`;
GRANT USAGE ON SCHEMA rearc_quest.lakehouse TO `ro.azure.1985_gmail.com#EXT#@roazure1985gmail.onmicrosoft.com`;
GRANT CREATE, MODIFY ON SCHEMA rearc_quest.lakehouse TO `ro.azure.1985_gmail.com#EXT#@roazure1985gmail.onmicrosoft.com`;


In [0]:
%sql
Show grants on catalog rearc_quest;

Principal,ActionType,ObjectType,ObjectKey
ro.azure.1985_gmail.com#ext#@roazure1985gmail.onmicrosoft.com,USE CATALOG,CATALOG,rearc_quest


In [0]:
%sql
Show grants on schema rearc_quest.lakehouse;

Principal,ActionType,ObjectType,ObjectKey
ro.azure.1985_gmail.com#ext#@roazure1985gmail.onmicrosoft.com,CREATE FUNCTION,SCHEMA,rearc_quest.lakehouse
ro.azure.1985_gmail.com#ext#@roazure1985gmail.onmicrosoft.com,CREATE TABLE,SCHEMA,rearc_quest.lakehouse
ro.azure.1985_gmail.com#ext#@roazure1985gmail.onmicrosoft.com,MODIFY,SCHEMA,rearc_quest.lakehouse
ro.azure.1985_gmail.com#ext#@roazure1985gmail.onmicrosoft.com,USE SCHEMA,SCHEMA,rearc_quest.lakehouse


# Catalog + Schema whose managed tables go to /tables/

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS rearc_quest
MANAGED LOCATION 'abfss://rearc-quest@rohitstorage1985.dfs.core.windows.net/tables/lakehouse/'
COMMENT 'Rearc Quest UC catalog';

CREATE SCHEMA IF NOT EXISTS rearc_quest.lakehouse
COMMENT 'Managed tables stored under /tables (no overlap with /data)';

# Volumes under /data/

In [0]:
%sql
CREATE EXTERNAL VOLUME IF NOT EXISTS rearc_quest.lakehouse.raw_bls
LOCATION 'abfss://rearc-quest@rohitstorage1985.dfs.core.windows.net/data/raw/bls'
COMMENT 'Raw BLS sync files';

In [0]:
%sql

CREATE EXTERNAL VOLUME IF NOT EXISTS rearc_quest.lakehouse.raw_datausa
LOCATION 'abfss://rearc-quest@rohitstorage1985.dfs.core.windows.net/data/raw/datausa'
COMMENT 'Raw DataUSA JSON files';


In [0]:
%sql
SHOW CATALOGS;

catalog
hive_metastore
rearc_quest
rohit1985_dbx
rohit_managed
samples
system


In [0]:
%sql
SHOW SCHEMAS IN rearc_quest;

databaseName
default
information_schema
lakehouse


In [0]:
%sql
SHOW TABLES IN rearc_quest.lakehouse;

database,tableName,isTemporary
lakehouse,bls_best_year_by_series,false
lakehouse,population_stats_2013_2018,false
lakehouse,report_prs30006032_q01,false


In [0]:
%sql
DESCRIBE SCHEMA EXTENDED rearc_quest.lakehouse;

database_description_item,database_description_value
Catalog Name,rearc_quest
Namespace Name,lakehouse
Comment,Managed tables stored under /tables (no overlap with /data)
Collation,UTF8_BINARY
Location,
Owner,ro.azure.1985@gmail.com
Properties,
Predictive Optimization,ENABLE (inherited from METASTORE metastore_azure_eastus)
